<a href="https://colab.research.google.com/github/EAName/yelp-reviews/blob/main/Yelp_Top_Words_for_Each_Business_RA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import random
import numpy as np
import pickle 
import re,string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, silhouette_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC

import xgboost as xgb

from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_predict


import seaborn as sns
import matplotlib.pyplot as plt

from gensim.models import Word2Vec,LdaMulticore, TfidfModel, CoherenceModel
from gensim import corpora
from gensim import similarities
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import LsiModel,LdaModel

from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, GRU
from keras.callbacks import EarlyStopping
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from keras import Input

from scipy import sparse

from IPython.display import display

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Load Reviews Dataset

In [ ]:
df =pd.read_csv('/content/gdrive/MyDrive/Data/yelp_reviews_clean_version4_st_louis.csv')

In [ ]:
df['stars'].value_counts()

5.0    124247
4.0     73648
3.0     35345
1.0     32873
2.0     26263
Name: stars, dtype: int64

In [ ]:
df = df[df['stars'].isin([1.0, 4.0, 5.0])] 

In [ ]:
df['stars'].value_counts()

5.0    124247
4.0     73648
1.0     32873
Name: stars, dtype: int64

In [ ]:
def clean_doc(doc): 
    #split review into individual words
    tokens=doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]

    #word lemmzatization    
    lemm = WordNetLemmatizer()
    tokens=[lemm.lemmatize(word) for word in tokens]

    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 3]
    #lowercase all words
    tokens = [word.lower() for word in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]     

    # filter out common words
    tokens = [w for w in tokens if not w in ['louis', 'restaurant', 'also', 'youre']] 
    # # word stemming    
    # ps=PorterStemmer()
    # tokens=[ps.stem(word) for word in tokens]

    return tokens

def final_processed_text(doc):
    #this is a function to join the processed text back
    ' '.join(doc)
    return doc

In [ ]:
df['cleaned_text'] = df['text'].apply(lambda x: clean_doc(x))

final_processed_text = [' '.join(x) for x in df['cleaned_text'].tolist()]
df['final_processed_text'] = final_processed_text     #add to dataframe

## Top Negative Words

In [ ]:
df_negative = df[df['stars'].isin([1.0])] 

In [ ]:
len(df_negative)

32873

In [ ]:
titles = list(range(len(df_negative)))
cv = CountVectorizer(binary=False, ngram_range = (1,1), max_features = 2000)
count_matrix = cv.fit_transform(df_negative['final_processed_text'])

words = cv.get_feature_names()
matrix=pd.DataFrame(count_matrix.toarray(), columns=words, index=titles)

In [ ]:
df_negative['business_id'].nunique()

2859

In [ ]:
# add business_id to count_matrix
business_negative = df_negative['business_id'].tolist()
matrix['business_id'] = business_negative

In [ ]:
# sum of words for each business
df_negative_sum = matrix.groupby('business_id').sum()

In [ ]:
len(df_negative_sum)

2859

In [ ]:
business_id_list = []
sentiment_list = []
words_list = []
count_list = []

for index, row in df_negative_sum.iterrows():
  sorted_words = row.sort_values(ascending=False)
  # print(sorted_words)
  # print(sorted_words[0])
  # print(sorted_words.index[0])
  for i in range(1,16):
    business_id_list.append(index)
    sentiment_list.append('negative')
    words_list.append(sorted_words.index[i])
    count_list.append(sorted_words[i])

In [ ]:
df_negative_top_words = pd.DataFrame(list(zip(business_id_list, sentiment_list, words_list, count_list)),
               columns =['business_id', 'sentiment', 'word', 'count'])

In [ ]:
df_negative_top_words

,business_id,sentiment,word,count
0,-1wnOZFsvsJPEXh_hWcmMA,negative,ordered,2
1,-1wnOZFsvsJPEXh_hWcmMA,negative,server,2
2,-1wnOZFsvsJPEXh_hWcmMA,negative,finally,2
3,-1wnOZFsvsJPEXh_hWcmMA,negative,food,2
4,-1wnOZFsvsJPEXh_hWcmMA,negative,minute,1
...,...,...,...,...
42880,zymvTQ12tRy3bzmP4tPgOg,negative,said,8
42881,zymvTQ12tRy3bzmP4tPgOg,negative,asked,7
42882,zymvTQ12tRy3bzmP4tPgOg,negative,charge,7
42883,zymvTQ12tRy3bzmP4tPgOg,negative,minute,7


## Top Positive Words

In [ ]:
df = df[df['stars'].isin([4.0, 5.0])] 

In [ ]:
len(df)

197895

In [ ]:
titles = list(range(len(df)))
cv = CountVectorizer(binary=False, ngram_range = (1,1), max_features = 2000)
count_matrix = cv.fit_transform(df['final_processed_text'])

words = cv.get_feature_names()
matrix=pd.DataFrame(count_matrix.toarray(), columns=words, index=titles)

In [ ]:
df['business_id'].nunique()

3163

In [ ]:
# add business_id to count_matrix
business_positive = df['business_id'].tolist()
matrix['business_id'] = business_positive

In [ ]:
# sum of words for each business
df_positive_sum = matrix.groupby('business_id').sum()

In [ ]:
len(df_positive_sum)

3163

In [ ]:
for index, row in df_positive_sum.iterrows():
  sorted_words = row.sort_values(ascending=False)
  # print(sorted_words)
  # print(sorted_words[0])
  # print(sorted_words.index[0])
  for i in range(1,16):
    business_id_list.append(index)
    sentiment_list.append('positive')
    words_list.append(sorted_words.index[i])
    count_list.append(sorted_words[i])

In [ ]:
df_top_words = pd.DataFrame(list(zip(business_id_list, sentiment_list, words_list, count_list)),
               columns =['business_id', 'sentiment', 'word', 'count'])

In [ ]:
df_top_words

,business_id,sentiment,word,count
0,-1wnOZFsvsJPEXh_hWcmMA,negative,ordered,2
1,-1wnOZFsvsJPEXh_hWcmMA,negative,server,2
2,-1wnOZFsvsJPEXh_hWcmMA,negative,finally,2
3,-1wnOZFsvsJPEXh_hWcmMA,negative,food,2
4,-1wnOZFsvsJPEXh_hWcmMA,negative,minute,1
...,...,...,...,...
90325,zzO2zgfqP9ANmEWt-EZFWg,positive,husband,9
90326,zzO2zgfqP9ANmEWt-EZFWg,positive,salad,9
90327,zzO2zgfqP9ANmEWt-EZFWg,positive,time,9
90328,zzO2zgfqP9ANmEWt-EZFWg,positive,perfectly,9


In [ ]:
df_top_words.isnull().sum()

business_id    0
sentiment      0
word           0
count          0
dtype: int64

In [ ]:
df_top_words.to_csv('/content/gdrive/MyDrive/Data/top_words_by_business_v2.csv', index = False)